In [1]:
import pandas as pd 
import numpy as np
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn-white')
from listas import North_America, Cen_Sth_America, Europe, Eurasia, Africa, Middle_East, Asia_Oceania
from variables import dicc_country, ltW, cdW
pd.set_option('display.max_rows', 45)

# Access to electricity (% of population)

df_E = pd.read_csv('WDIData.csv', sep=',', engine='python', index_col=False)

''' Filtramos nuestro DataSet'''

# df_P1 = df_E[df_E['Country Name'].isin(ltW)]
# df_P1['Country Name'].nunique()
df_E = df_E[df_E['Country Code'].isin(cdW)]

'''Escogemos únicamente el dato que nos interesa y
 entre el espacio temporal que estudiamos'''

df_AE = df_E[df_E['Indicator Name'] == 'Access to electricity (% of population)']
df_AE = df_AE.drop(df_AE.loc[:,'1960':'1979'], axis=1)

'''Limpiamos por encima nuestro DataSet para que quede 
equivalente a loos demás'''

df_AE = df_AE.drop(df_AE.loc[:,'Indicator Name':'Indicator Code'], axis=1)
df_AE.reset_index(drop=True, inplace=True)
df_AE.rename(columns={'Country Name': 'Country', 'Country Code': 'Code'}, inplace=True)

'''Nos deshacemos de las olumnas totalmente vacías y
creamos dos DataSets por separado
1) Uno más completo pero con Nan
2) Otro que abarco un período menor pero completamente limpio de Nan'''

df_AE = df_AE.dropna(how='all', axis=1)
df_AE = df_AE.dropna(thresh=3, axis=0).reset_index(drop=True)
dfAE = df_AE.dropna(axis= 1).reset_index(drop=True)


In [2]:
df_AE.shape, dfAE.shape

((204, 32), (204, 11))

In [3]:
df_AE.head()

,"ï»¿""Country Name""",Code,1990,1991,1992,1993,1994,1995,1996,1997,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Afghanistan,AFG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,42.700000,43.222019,69.100000,68.982941,89.500000,71.50000,97.700000,97.700000,98.715622,97.700000
1,Albania,ALB,100.000000,100.000000,100.000000,100.000000,100.00000,100.000000,100.000000,100.000000,...,100.000000,100.000000,99.900000,100.000000,99.950000,99.98000,99.890000,99.890000,100.000000,100.000000
2,Algeria,DZA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,98.821838,98.847992,98.764660,98.977257,99.074036,99.18689,99.402870,99.615135,99.641922,99.500000
3,Angola,AGO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,34.879726,34.600000,37.101025,38.248993,32.000000,42.00000,41.797379,43.001610,45.290000,45.670315
4,Antigua and Barbuda,ATG,95.763046,95.940651,96.118134,96.294945,96.47039,96.643799,96.814484,96.981773,...,98.600403,94.552014,100.000000,100.000000,100.000000,100.00000,100.000000,100.000000,99.999725,100.000000


## Cálculo de Relative Change

In [4]:
# Cuidado!! Esta vez no hemos hecho esta conversión
# df_E2 = df_E1.fillna(0)
# df_E1[df_E1['Country Name'] == 'Chad'][['2000', '2018']]
# p1 = (df_E1[df_E1['Country Name'] == 'Chad']['2018'] - df_E1[df_E1['Country Name'] == 'Chad']['2000']) / df_E1[df_E1['Country Name'] == 'Chad']['2000']

In [5]:
'''
Hay que saber interpretar el resultado:
- Si el coeficiente sale 0 , seguramente no haya existido una evolución entre el periodo estudiado
- Si el coeficiente inf, solo existe uno de los pparamatros necesarios para lo operación o el coeficiente es negativo
- Si el coeficiente indica "float64%" no existe datos para resolver la operación
'''

#He calculado el coeficiente de progreso desde el año 1998 a 2018

def relative_change(df, anio_M, anio_m):
    locura = []
    for pais in df['Country'].unique():
        n1 = df[df['Country'] == pais][anio_M] - df[df['Country'] == pais][anio_m]
        n2 = df[df['Country'] == pais][anio_m]
        n3 = round(((n1/n2) * 100) , 3)
        dato = str(n3)
        locura.append(dato)
    # return locura
    
    lst_182= []
    for num in locura:
        txt = num.split(' ')
        # print(txt)
        txt2 = txt[4]
        # print(txt2)
        txt3 = txt2.split('\ndtype:')
        # print(txt3)
        txt4 = txt3[0]
        # if txt4 == 'float64':
        #    txt4 = txtX[X]
        lst_182.append(txt4)

    return pd.Series(lst_182)

ds1 = relative_change(dfAE,'2019', '2011')
 
'''Estos países tienen una desarrollo negativo'''

ds1[36] = '-6.459'
ds1[107] = '-14.33'
ds1[135] = '-0.894'
ds1[157]= '-0.2'
ds1[176]= '-2.483'

# pd.to_numeric(ds1) # AE['Relative Change (%)'] = ds1
dfAE['Relative Change (%)'] = pd.to_numeric(ds1, errors ='coerce')

KeyError: 'Country'

In [ ]:
dfAE.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204 entries, 0 to 203
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Country              204 non-null    object 
 1   Code                 204 non-null    object 
 2   2011                 204 non-null    float64
 3   2012                 204 non-null    float64
 4   2013                 204 non-null    float64
 5   2014                 204 non-null    float64
 6   2015                 204 non-null    float64
 7   2016                 204 non-null    float64
 8   2017                 204 non-null    float64
 9   2018                 204 non-null    float64
 10  2019                 204 non-null    float64
 11  Relative Change (%)  204 non-null    float64
dtypes: float64(10), object(2)
memory usage: 19.2+ KB


In [ ]:
AE.head()

,Country,Code,2011,2012,2013,2014,2015,2016,2017,2018,2019,Relative Change (%)
0,Afghanistan,AFG,43.222019,69.100000,68.982941,89.500000,71.50000,97.700000,97.700000,98.715622,97.700000,126.042
1,Albania,ALB,100.000000,99.900000,100.000000,99.950000,99.98000,99.890000,99.890000,100.000000,100.000000,0.000
2,Algeria,DZA,98.847992,98.764660,98.977257,99.074036,99.18689,99.402870,99.615135,99.641922,99.500000,0.660
3,Angola,AGO,34.600000,37.101025,38.248993,32.000000,42.00000,41.797379,43.001610,45.290000,45.670315,31.995
4,Antigua and Barbuda,ATG,94.552014,100.000000,100.000000,100.000000,100.00000,100.000000,100.000000,99.999725,100.000000,5.762


In [ ]:
AE.shape

(204, 12)

In [ ]:
AE.describe().T

,count,mean,std,min,25%,50%,75%,max
2011,204.0,79.649992,29.681707,2.689738,62.338485,98.843996,100.000,100.000
2012,204.0,80.484103,28.992297,3.137245,63.561147,99.110130,100.000,100.000
2013,204.0,81.022866,28.750688,3.608862,68.176149,99.122673,100.000,100.000
2014,204.0,81.794998,28.147660,4.100573,70.588238,99.333618,100.000,100.000
2015,204.0,82.444772,27.547626,4.608357,70.771019,99.419811,100.000,100.000
2016,204.0,83.560240,26.581488,5.128197,74.918388,99.656021,100.000,100.000
2017,204.0,84.339489,25.982980,4.200000,78.895229,99.800000,100.000,100.000
2018,204.0,85.102637,25.156980,6.187970,78.827359,99.899524,100.000,100.000
2019,204.0,85.686919,24.913049,6.720535,80.880117,100.000000,100.000,100.000
Relative Change (%),204.0,20.047206,52.768889,-14.330000,0.000000,0.789000,20.704,574.371


In [ ]:
'''BASURA'''

# ds1.replace('float64%', 'inf', inplace=True)
# ds1 = pd.to_numeric(ds1[0])
# ds1.replace(float('inf'), np.nan, inplace=True)

# ds2.replace('float64%', 'inf', inplace=True)
# ds2 = pd.to_numeric(ds2[0])
# ds2.replace(float('inf'), np.nan, inplace=True)

# '''No se porque no funciona!'''
# # ds1.update(ds2, overwrite = True)
# ds1.fillna(ds2)
# ds1.fillna(ds3)
# ds1.fillna(ds4)

'BASURA'

## Hay que estudiar África Outliers y casos interesantes

In [ ]:
AE[AE['Relative Change (%)'] > 50].sort_values('Relative Change (%)', ascending=False)

,Country,Code,2011,2012,2013,2014,2015,2016,2017,2018,2019,Relative Change (%)
106,Liberia,LBR,4.100000,9.128645,9.800000,9.400000,15.527711,17.700000,24.200000,24.818079,27.649210,574.371
152,Rwanda,RWA,10.800000,17.500000,15.200000,19.800000,22.800000,29.370000,34.100000,34.742237,37.782436,249.837
189,Uganda,UGA,14.600000,20.374794,13.900000,20.400000,18.500000,26.700000,32.724560,42.700000,41.300000,182.877
178,Tanzania,TZA,14.200000,15.300000,16.400000,23.500000,26.342781,32.800000,32.418404,35.229450,37.700000,165.493
166,South Sudan,SSD,2.689738,3.137245,3.608862,4.100573,4.608357,5.128197,4.200000,6.187970,6.720535,149.858
78,Guinea-Bissau,GNB,13.379713,14.778109,16.200615,17.200000,20.141108,23.049215,26.000000,28.514830,31.040413,131.996
0,Afghanistan,AFG,43.222019,69.100000,68.982941,89.500000,71.500000,97.700000,97.700000,98.715622,97.700000,126.042
60,Ethiopia,ETH,23.000000,35.202663,36.161236,27.200000,29.000000,42.900000,44.300000,45.053402,48.271622,109.877
142,Papua New Guinea,PNG,31.547920,33.690975,35.858147,39.459965,44.360016,49.400000,54.400000,58.887634,63.464512,101.169
105,Lesotho,LSO,22.377247,20.560000,26.733582,27.800000,31.784077,35.181511,33.700000,47.000000,44.640678,99.491


In [ ]:
'''Necesito aprender a situarlos en el mapa'''

'Necesito aprender a situarlos en el mapa'

In [ ]:
AE[AE['Code'] == 'UKR']

,Country,Code,2011,2012,2013,2014,2015,2016,2017,2018,2019,Relative Change (%)
190,Ukraine,UKR,100.0,99.867491,100.0,100.0,100.0,100.0,100.0,100.0,100.0,0.0


In [ ]:
AE[AE['2019'] < 70][['Country','2011','2019', 'Relative Change (%)']].sort_values('2019', ascending=True).reset_index(drop=True)

,Country,2011,2019,Relative Change (%)
0,South Sudan,2.689738,6.720535,149.858
1,Chad,8.980000,8.400000,-6.459
2,Burundi,6.106936,11.064797,81.184
3,Malawi,7.600000,11.200000,47.368
4,Central African Republic,10.310723,14.300000,38.691
5,Burkina Faso,14.899405,18.379152,23.355
6,Niger,14.300000,18.774725,31.292
7,"Congo, Dem. Rep.",13.535879,19.100000,41.106
8,Sierra Leone,14.200000,22.700000,59.859
9,Madagascar,14.300000,26.907185,88.162


In [ ]:
AE[AE['Country'] == 'Botswana']

,Country,Code,2011,2012,2013,2014,2015,2016,2017,2018,2019,Relative Change (%)
23,Botswana,BWA,53.24,55.874313,57.890686,59.940395,62.13,64.295151,67.4,68.358658,70.183182,31.824


In [ ]:
df_Afr = AE[AE['Country'].isin(Africa)][['Country', 'Code', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', 'Relative Change (%)']]
df_Afr.sort_values('2012', ascending=False).reset_index(drop=True)[0:5]

,Country,Code,2012,2013,2014,2015,2016,2017,2018,2019,Relative Change (%)
0,Tunisia,TUN,99.500000,99.700000,99.800000,99.900000,100.00000,100.000000,99.800000,100.000000,0.503
1,Algeria,DZA,98.764660,98.977257,99.074036,99.186890,99.40287,99.615135,99.641922,99.500000,0.660
2,Morocco,MAR,94.148445,97.200000,91.600000,97.300000,97.80000,100.000000,98.100000,99.600000,6.659
3,Gabon,GAB,89.300000,86.400000,86.989952,87.329674,88.16037,86.270000,89.841843,90.685257,2.034
4,South Africa,ZAF,85.300000,85.200000,86.000000,85.300000,84.20000,84.400000,84.700000,85.000000,1.675


In [ ]:
lst_Afr = ['TUN', 'DZA', 'MAR', 'GAB', 'ZAF']